In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("updated_dataset.csv")
df.head()

,distance,surge_multiplier,price,Black,Black SUV,Lux,Lux Black,Lux Black XL,Lyft,Lyft XL,Shared,UberPool,UberX,UberXL,WAV,Lyft.1,Uber
0,0.44,1.0,5.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,0.44,1.0,11.0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,0.44,1.0,7.0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,0.44,1.0,26.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0.44,1.0,9.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [4]:
from sklearn.preprocessing import PolynomialFeatures


selected_columns = ['surge_multiplier','distance']


degree = 5


poly_features = PolynomialFeatures(degree=degree)


poly_transform = poly_features.fit_transform(df[selected_columns])


poly_feature_names = poly_features.get_feature_names(selected_columns)


poly_df = pd.DataFrame(poly_transform, columns=poly_feature_names)


df_poly = pd.concat([df.drop(columns=selected_columns), poly_df], axis=1)

df_poly

c:\Users\VEDANT\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,price,Black,Black SUV,Lux,Lux Black,Lux Black XL,Lyft,Lyft XL,Shared,UberPool,...,surge_multiplier^3 distance,surge_multiplier^2 distance^2,surge_multiplier distance^3,distance^4,surge_multiplier^5,surge_multiplier^4 distance,surge_multiplier^3 distance^2,surge_multiplier^2 distance^3,surge_multiplier distance^4,distance^5
0,5.0,0,0,0,0,0,0,0,1,0,...,0.44,0.1936,0.085184,0.037481,1.0,0.44,0.1936,0.085184,0.037481,0.016492
1,11.0,0,0,1,0,0,0,0,0,0,...,0.44,0.1936,0.085184,0.037481,1.0,0.44,0.1936,0.085184,0.037481,0.016492
2,7.0,0,0,0,0,0,1,0,0,0,...,0.44,0.1936,0.085184,0.037481,1.0,0.44,0.1936,0.085184,0.037481,0.016492
3,26.0,0,0,0,0,1,0,0,0,0,...,0.44,0.1936,0.085184,0.037481,1.0,0.44,0.1936,0.085184,0.037481,0.016492
4,9.0,0,0,0,0,0,0,1,0,0,...,0.44,0.1936,0.085184,0.037481,1.0,0.44,0.1936,0.085184,0.037481,0.016492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632382,9.5,0,0,0,0,0,0,0,0,0,...,1.00,1.0000,1.000000,1.000000,1.0,1.00,1.0000,1.000000,1.000000,1.000000
632383,13.0,0,0,0,0,0,0,0,0,0,...,1.00,1.0000,1.000000,1.000000,1.0,1.00,1.0000,1.000000,1.000000,1.000000
632384,9.5,0,0,0,0,0,0,0,0,0,...,1.00,1.0000,1.000000,1.000000,1.0,1.00,1.0000,1.000000,1.000000,1.000000
632385,27.0,0,1,0,0,0,0,0,0,0,...,1.00,1.0000,1.000000,1.000000,1.0,1.00,1.0000,1.000000,1.000000,1.000000


In [5]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
X = df_poly.drop(columns=['price'])  
y = df_poly['price']  

In [7]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


lasso_regressor = Pipeline([
    ('scaler', StandardScaler()),  
    ('lasso', Lasso(alpha=0.1, random_state=42))  
])


lasso_regressor.fit(X_train, y_train)


y_pred = lasso_regressor.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 5.186949544324744


In [8]:
from sklearn.metrics import r2_score,mean_absolute_percentage_error
score = r2_score(y_test,y_pred)
print("R2Score",score)
mape = mean_absolute_percentage_error(y_test,y_pred)
print("Mean Abs Percent Error",mape)

R2Score 0.9325324918106952
Mean Abs Percent Error 0.12533624091628495


In [9]:
y_train_pred = lasso_regressor.predict(X_train)
r2_score(y_train,y_train_pred)

0.9326123181606634

R2 Score: 0.9325

Maximum Abs Percent Error = 12.53%